In [2]:
import pandas as pd
!pip install SQLAlchemy
!pip install pymysql
from sqlalchemy import create_engine
import pymysql
import configparser
import matplotlib
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import make_pipeline,Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

In [101]:
## read data from ctrdata.csv
Origin = pd.read_csv('test3_data.csv')
Origin = Origin.drop(columns='Unnamed: 0')
Origin ['Action'] [Origin['Action'] == 'Click']  = 1
Origin ['Action'][Origin['Action'] == 'View'] = 0

<ipython-input-101-35cce4bd7539>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Origin ['Action'] [Origin['Action'] == 'Click']  = 1
<ipython-input-101-35cce4bd7539>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Origin ['Action'][Origin['Action'] == 'View'] = 0


In [111]:
## Decide output, split testdata
from sklearn.model_selection import train_test_split
X = Origin.drop(columns=['Action'])
y = Origin['Action']

adX_train, adX_test, ady_train, ady_test = train_test_split(X, y, test_size=0.25)

## Feature engineering from ctrX_train

## fillna
ITmean = adX_train['InteractionTime'].mean() 
adX_train['InteractionTime'] = adX_train['InteractionTime'].fillna(ITmean)
#datetime engineering 
adX_train['ActionTime']=pd.to_datetime(adX_train['ActionTime'].astype('float64')/10**6, unit='s')

hr = pd.to_datetime(adX_train ['ActionTime']).dt.hour
time = [0, 2, 6, 9, 11,19, 24]
time_name = ['late_night', 'early_morning', 'morning', 'noon', 'afternoon', 'evening']
adX_train['Daysection'] = pd.cut(hr, bins=time, labels=time_name)

def wd(a):
    if a==5 or a==6:
        b='weekend'
    else:
        b='weekday'
    return b

adX_train['Dayofweek'] = pd.to_datetime(adX_train ['ActionTime']).dt.dayofweek.apply(wd)

# Banner Size
adX_train['BannerSize'] = adX_train['BannerSize'].str.extract(r'(\d+)x').fillna(0).astype(int) * adX_train['BannerSize'].str.extract(r'x(\d+)').fillna(0).astype(int)
AvgBan = adX_train['BannerSize'].mean()
adX_train['BannerSize'][adX_train['BannerSize'] == 0] = AvgBan

#Threshold Capping BannerSize, InteractionTime
def TCap(columns):
    for i in columns:
        q1, q3 = adX_train[i].quantile([0.25, .75])

        iqr = q3 - q1

        top_boundary = q3 + (iqr * 1.5)

        adX_train.loc[adX_train[i] >= top_boundary,[i]] = top_boundary
TCap_columns=['BannerSize', 'InteractionTime' ]
TCap(TCap_columns)
     
## arrange inputs and outputs
unusedcolumns = []
X_input = adX_train[['ID', 'Website', 'BannerSize', 'Brand', 'colour',
       'InteractionTime', 'Daysection', 'Dayofweek']]

y_input = ady_train
y_test = ady_test
     
def FeatureEngineering(X):
    ITmean = X['InteractionTime'].mean() 
    X['InteractionTime'] = X['InteractionTime'].fillna(ITmean)
    #datetime engineering 
    X['ActionTime']=pd.to_datetime(X['ActionTime'].astype('float64')/10**6, unit='s')

    hr = pd.to_datetime(X['ActionTime']).dt.hour
    time = [0, 2, 6, 9, 11,19, 24]
    time_name = ['late_night', 'early_morning', 'morning', 'noon', 'afternoon', 'evening']
    X['Daysection'] = pd.cut(hr, bins=time, labels=time_name)

    def wd(a):
        if a==5 or a==6:
            b='weekend'
        else:
            b='weekday'
        return b

    X['Dayofweek'] = pd.to_datetime(X['ActionTime']).dt.dayofweek.apply(wd)

    # Banner Size
    X['BannerSize'] = X['BannerSize'].str.extract(r'(\d+)x').fillna(0).astype(int) * X['BannerSize'].str.extract(r'x(\d+)').fillna(0).astype(int)
    AvgBan = X['BannerSize'].mean()
    X['BannerSize'][X['BannerSize'] == 0] = AvgBan

    #Threshold Capping BannerSize, InteractionTime
    def TCap(columns):
        for i in columns:
            q1, q3 = X[i].quantile([0.25, .75])

            iqr = q3 - q1

            top_boundary = q3 + (iqr * 1.5)

            X.loc[X[i] >= top_boundary,[i]] = top_boundary
    TCap_columns=['BannerSize', 'InteractionTime' ]
    TCap(TCap_columns)
    X = X[['ID', 'Website', 'BannerSize', 'Brand', 'colour',
       'InteractionTime', 'Daysection', 'Dayofweek']]
    return X
X_test = FeatureEngineering(adX_test)  


## inputs and test sets
[X_input.shape, X_test.shape, y_input.shape, y_test.shape]

<ipython-input-111-fe5b2089c0ce>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adX_train['InteractionTime'] = adX_train['InteractionTime'].fillna(ITmean)
<ipython-input-111-fe5b2089c0ce>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adX_train['ActionTime']=pd.to_datetime(adX_train['ActionTime'].astype('float64')/10**6, unit='s')
<ipython-input-111-fe5b2089c0ce>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [117]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier()),
    ('lr', LogisticRegression()),
    ('xgb', XGBClassifier(max_depth=3,min_child_weight=1.5))
]

clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

In [121]:
from sklearn.preprocessing import OneHotEncoder
ros = RandomOverSampler(random_state=42)
##X_resampled_click, y_resampled_click = ros.fit_resample(X_input_click, y_input_click)
from sklearn.decomposition import PCA

## use GridSearch to tune the hyperparameters
pipe = Pipeline([
        ('OHE', OneHotEncoder()),
      ##('Smote',SMOTE()),
        ('o', ros), 
        ##('u', rus),
     ('scaling', MinMaxScaler()),
     ##('scaling', RobustScaler()),
     ##('scaling', StandardScaler()),
    ('simpleimputer', SimpleImputer()),
    ##('DT', DecisionTreeClassifier())
    ##('LR', LogisticRegression())
    ('PCA', PCA(0.9)),
    ('clf', clf)
    
        
])

parameters = {
    'o__sampling_strategy':[0.8],
    'clf__dt__max_depth':  [50, 100],
    'clf__knn__n_neighbors': [3, 5, 7],
    'clf__lr__penalty':  ['l1', 'l2'],
    'clf__lr__C':  [20.0, 5.0, 1.0],
    'clf__xgb__max_depth':  [1, 3, 5],
    ##'LR__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga' ]
}


gs = GridSearchCV(pipe, parameters, cv=10, scoring='f1', n_jobs=2, verbose=4)

gs.fit(X_input, y_input)
## not enough time to finish running this

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


KeyboardInterrupt: 

In [ ]:
gs.best_params_



In [ ]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))